In [122]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from math import sqrt

In [102]:
data = pd.read_json("..\Datasets\WDO@_M5.json")
data['time'] = pd.to_datetime(data['time'], unit='ms')
columns = ['time', 'open', 'high', 'low', 'close']
data = data.loc[(data['time']>='2021-03-1 8:00:00')&(data['time']<='2021-03-5 19:00:00'), columns]
data.reset_index(inplace=True, drop=True)
data.tail()

,time,open,high,low,close
559,2021-03-05 18:05:00,5686.5,5689.0,5683.0,5683.5
560,2021-03-05 18:10:00,5683.0,5698.5,5682.5,5698.5
561,2021-03-05 18:15:00,5697.0,5700.0,5692.0,5699.0
562,2021-03-05 18:20:00,5699.0,5703.5,5695.5,5701.0
563,2021-03-05 18:25:00,5701.5,5703.5,5696.0,5697.0


In [135]:
total_sum=0
for idx in range(data.shape[0]):
    total_sum+=data.iloc[idx]['high']-data.iloc[idx]['low']
mean = total_sum/data.shape[0]
print("mean: ", mean)

total_sum=0
for idx in range(data.shape[0]):
    total_sum+=(mean-(data.iloc[idx]['high']-data.iloc[idx]['low']))**2

bar_std = sqrt(total_sum/data.shape[0])
print("std: ", bar_std)

mean:  10.955673758865249
std:  6.457085810964763


In [131]:
total_sum**total_sum

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_14460/4235582957.py:1: RuntimeWarning:

overflow encountered in double_scalars



inf

In [130]:
bigger_than=data.loc[(abs(data['open']-data['close'])>=mean)]
fig = go.Figure(data=[go.Candlestick(x=bigger_than['time'],
                open=bigger_than['open'],
                high=bigger_than['high'],
                low=bigger_than['low'],
                close=bigger_than['close'])])


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[18, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [103]:
fig = go.Figure(data=[go.Candlestick(x=data['time'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[18, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [107]:
depth=backstep=3

def zig_zag():
    highs = np.full(data.shape[0],None)
    lows = np.full(data.shape[0],None)
    last_high=data.iloc[0]['high']
    last_low=data.iloc[0]['low']

    for idx, _ in data.iterrows():
        if idx < depth:
            continue
        max = np.amax(data[idx-depth:idx]['high'])
        min = np.amin(data[idx-depth:idx]['low'])
        if max == data.iloc[idx-1]['high'] and last_high!=max:
            highs[idx-1]=max
            last_high=max
        if min == data.iloc[idx-1]['low'] and min!=last_low:
            lows[idx-1]=min
            last_low=min
        
        for b in range(2, backstep+1):
            if lows[idx-1]!=None and lows[idx-b]!=None and lows[idx-b]>lows[idx-1]:
                lows[idx-b]=None
            if highs[idx-1]!=None and highs[idx-b]!=None and highs[idx-b]<highs[idx-1]:
                highs[idx-b]=None

    zz = np.full(data.shape[0], None)

    today=0
    for idx in range(0, data.shape[0]):
        
        if data.iloc[idx]['time'].day != today:
            zz[idx]=data.iloc[idx]['high']
            today=data.iloc[idx]['time'].day

        if highs[idx] != None:
            zz[idx]=highs[idx]
            
            for last in reversed(range(idx)):
                if zz[last] != None:
                    if zz[last] != highs[last]:
                        break
                    if zz[last] == highs[last]:
                        if zz[last]>=zz[idx]:
                            zz[idx]=None
                            break
                        else:
                            zz[last]=None
                            break

        if lows[idx] != None:
            zz[idx]=lows[idx]

            for last in reversed(range(idx)):
                if zz[last] != None:
                    if zz[last] != lows[last]:
                        break
                    if zz[last] == lows[last]:
                        if zz[last]<=zz[idx]:
                            zz[idx]=None
                            break
                        else:
                            zz[last]=None
                            break
        
    data['zz']=zz
    data['lows']=lows
    data['highs']=highs

In [108]:
zig_zag()

In [109]:
fig = go.Figure(data=[go.Candlestick(x=data['time'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

fig.add_trace(go.Scatter(
    x=data.loc[(data['zz'].notnull())]['time'],
    y=data.loc[(data['zz'].notnull())]['zz']
))


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[19, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [110]:
leg_sizes=[]
high_idx=low_idx=None
for idx in range(data.shape[0]):
    if data.iloc[idx]['zz'] != None:
        if data.iloc[idx]['zz'] == data.iloc[idx]['high']:
            high_idx=idx
        if data.iloc[idx]['zz'] == data.iloc[idx]['low']:
            low_idx=idx
        if high_idx!=None and low_idx!=None:
            leg_sizes.append(dict(size=abs(data.iloc[high_idx]['zz']-data.iloc[low_idx]['zz']),
                            idx=[low_idx,high_idx]))
            if low_idx == idx:
                high_idx=None
            else:
                low_idx=None

print(leg_sizes)

[{'size': 26.0, 'idx': [4, 0]}, {'size': 13.5, 'idx': [4, 8]}, {'size': 13.0, 'idx': [10, 8]}, {'size': 32.5, 'idx': [10, 14]}, {'size': 46.0, 'idx': [23, 14]}, {'size': 42.5, 'idx': [23, 30]}, {'size': 49.5, 'idx': [41, 30]}, {'size': 25.0, 'idx': [41, 47]}, {'size': 17.0, 'idx': [61, 47]}, {'size': 37.5, 'idx': [61, 68]}, {'size': 11.0, 'idx': [71, 68]}, {'size': 17.0, 'idx': [71, 77]}, {'size': 12.5, 'idx': [78, 77]}, {'size': 8.5, 'idx': [78, 86]}, {'size': 12.5, 'idx': [90, 86]}, {'size': 50.5, 'idx': [90, 104]}, {'size': 11.0, 'idx': [107, 104]}, {'size': 40.5, 'idx': [107, 110]}, {'size': 16.5, 'idx': [112, 110]}, {'size': 38.5, 'idx': [112, 117]}, {'size': 27.0, 'idx': [121, 117]}, {'size': 30.0, 'idx': [121, 126]}, {'size': 23.0, 'idx': [128, 126]}, {'size': 17.0, 'idx': [128, 130]}, {'size': 26.0, 'idx': [134, 130]}, {'size': 47.0, 'idx': [134, 144]}, {'size': 18.5, 'idx': [146, 144]}, {'size': 41.0, 'idx': [146, 153]}, {'size': 47.5, 'idx': [157, 153]}, {'size': 34.0, 'idx':

In [136]:
total_size_sum=0
for idx in range(len(leg_sizes)):
    total_size_sum+=leg_sizes[idx]['size']

leg_mean=total_size_sum/len(leg_sizes)
print(leg_mean)

total_size_sum=0
for idx in range(len(leg_sizes)):
    total_size_sum+=(leg_mean-leg_sizes[idx]['size'])**2

leg_std = sqrt(total_size_sum/len(leg_sizes))
print(leg_std)

27.66417910447761
17.334265965564754


In [139]:
bigger_legs=list()
for idx in range(len(leg_sizes)):
    if leg_sizes[idx]['size']>leg_mean:
        bigger_legs.append(leg_sizes[idx]['idx'])

# bigger_legs=bigger_legs.sort()
print(bigger_legs)

[[10, 14], [23, 14], [23, 30], [41, 30], [61, 68], [90, 104], [107, 110], [112, 117], [121, 126], [134, 144], [146, 153], [157, 153], [157, 159], [178, 173], [193, 185], [193, 194], [200, 194], [223, 229], [246, 253], [260, 265], [269, 274], [304, 294], [304, 307], [322, 313], [322, 324], [327, 324], [327, 331], [332, 331], [338, 347], [351, 347], [357, 354], [366, 363], [366, 369], [371, 369], [371, 376], [380, 376], [383, 382], [383, 387], [390, 387], [397, 410], [414, 423], [425, 427], [431, 427], [431, 437], [447, 459], [466, 468], [471, 468], [478, 473], [478, 481], [491, 485], [491, 503], [510, 503]]


In [140]:


fig = go.Figure(data=[go.Candlestick(x=data['time'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])
bigger_legs = list(bigger_legs)

for idx in range(len(bigger_legs)):
    fig.add_shape(type="line",
        x0=data.loc[bigger_legs[idx][0]]['time'],
        y0=data.loc[bigger_legs[idx][0]]['zz'],
        x1=data.loc[bigger_legs[idx][1]]['time'],
        y1=data.loc[bigger_legs[idx][1]]['zz'],
        line=dict(color="black",width=4)
    )


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[19, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()